In [1]:
# run initial imports
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from fbprophet import Prophet
from fbprophet.plot import plot_plotly

import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go

from scipy.integrate import odeint
from scipy.optimize import curve_fit

from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets

import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

import os
from datetime import datetime

In [2]:
NOW = datetime.now()
input_dir = './data/'
state_geo = os.path.join('./reference/', 'us-states.json')
state_data = pd.read_csv('./reference/COVID19-US-State-Quarantine.csv')
combined = pd.read_csv(input_dir + 'web_cases.csv')

state_data['Quarantined'] = 1
state_data.loc[state_data['US States Quarantined'] == 2, 'Quarantined'] = 0

combined['Last Update'] = pd.to_datetime(combined['Last_Update'])
combined['Latitude'] = combined['Lat']
combined['Longitude'] = combined['Long_']
combined['Province/State'] = combined['Province_State']
combined['Country/Region'] = combined['Country_Region']
combined = combined.sort_values('Last Update').reset_index(drop=True)

#extract lat, lon for non mapped stuff
# locations = combined.groupby(['Country/Region', 'Province/State'])['Latitude', 'Longitude'].mean().reset_index()
# locations.columns = ['Country/Region', 'Province/State', 'Latitude_Lookup', 'Longitude_Lookup']

# combined = pd.merge(left=combined, right=locations, left_on=['Country/Region', 'Province/State'], right_on=['Country/Region', 'Province/State'], how='left')
# combined['Latitude'] = combined['Latitude'].fillna(combined['Latitude_Lookup'])
# combined['Longitude'] = combined['Longitude'].fillna(combined['Longitude_Lookup'])
# combined['Province/State'] = combined['Province/State'].fillna(combined['Country/Region'])
# del combined['Latitude_Lookup'] 
# del combined['Longitude_Lookup'] 
# combined = combined.fillna(0)

In [3]:
#do a little re-ordering
combined = combined[['Last Update', 'Latitude', 'Longitude', 'Country/Region', 'Province/State','Active', 'Confirmed','Recovered','Deaths']].copy().reset_index(drop=True)
#combined = combined[combined['Country/Region'] != 0]
#combined

In [4]:
latest = combined.sort_values('Last Update').groupby(['Latitude','Longitude']).tail(1).copy().reset_index(drop=True)
latest['Active'] = latest['Confirmed'] - latest['Recovered'] - latest['Deaths']
#latest

In [5]:
deaths = latest[latest['Deaths']>0]
deaths = pd.DataFrame(deaths.values.repeat(deaths.Deaths, axis=0), columns=deaths.columns)

confirmed = latest[latest['Confirmed']>0]
confirmed = pd.DataFrame(confirmed.values.repeat(confirmed.Confirmed, axis=0), columns=confirmed.columns)

us_confirmed = latest[latest['Confirmed']>0]
us_confirmed = us_confirmed[us_confirmed['Country/Region'] == 'US']
us_confirmed = pd.DataFrame(us_confirmed.values.repeat(us_confirmed.Confirmed, axis=0), columns=us_confirmed.columns)

recovered = latest[latest['Recovered']>0]
recovered = pd.DataFrame(recovered.values.repeat(recovered.Recovered, axis=0), columns=recovered.columns)

active = latest[latest['Active']>0]
active = pd.DataFrame(active.values.repeat(active.Active, axis=0), columns=active.columns)

us_active = latest[latest['Active']>0]
us_active = us_active[us_active['Country/Region'] == 'US']
us_active = pd.DataFrame(us_active.values.repeat(us_active.Active, axis=0), columns=us_active.columns)

na = latest[latest['Active'] > 0]
na_active = na[na['Country/Region'] == 'US']
na_active = na_active.append(na[na['Country/Region'] == 'Mexico'])
na_active = na_active.append(na[na['Country/Region'] == 'Canada'])
na_active = pd.DataFrame(na_active.values.repeat(na_active.Active, axis=0), columns=na_active.columns)

In [7]:
folium_map = folium.Map(location=[38.826555, -100.244867],
                        zoom_start=4,
                        tiles=None)
folium.TileLayer('CartoDB dark_matter', name='Base Layer').add_to(folium_map)
folium.TileLayer('openstreetmap', name='Openstreetmap').add_to(folium_map)
folium.TileLayer('Stamen Terrain', name='Terrain').add_to(folium_map)

folium.Choropleth(
    name='Quarantine in Effect',
    geo_data=state_geo,
    data=state_data,
    columns=['US States', 'Quarantined'],
    key_on='feature.properties.name',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2
).add_to(folium_map, index=10)

hm_data = combined[['Latitude', 'Longitude', 'Active']].copy()
hm_data_death = combined[['Latitude', 'Longitude', 'Deaths']].copy()


HeatMap(data=hm_data, name='Active Cases').add_to(folium_map, index=0)
HeatMap(data=hm_data_death, name='Deaths').add_to(folium_map, index=1)

lc = folium.LayerControl(collapsed=False).add_to(folium_map)

#FastMarkerCluster(data=list(zip(deaths['Latitude'].values, deaths['Longitude'].values)), name='Reported Deaths').add_to(folium_map)
#FastMarkerCluster(data=list(zip(recovered['Latitude'].values, recovered['Longitude'].values)), name='Recovered Cases').add_to(folium_map)
#FastMarkerCluster(data=list(zip(na_active['Latitude'].values, na_active['Longitude'].values)), name='Active Cases').add_to(folium_map)

# this approach is too slow with more than a state by state case
# alat=list(active.Latitude)
# alon=list(active.Longitude)
# alab=list(active.Active)
# mc = MarkerCluster()
# for lat, lng, label in zip(alat, alon, alab):
#     folium.Circle(
#         location = [lat, lng], 
#         radius=50,
#         color='crimson',
#         popup=str(label)+' Active Case(s)',
#         fill=True,
#         fill_color='crimson'
#      ).add_to(mc) 

# mc.add_to(folium_map)

#folium_map.save('./www/mapping.html')
folium_map